In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

In [3]:
import numpy as np
import pandas as pd
import json
import sys

import matplotlib.pyplot as plt

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'
n_models = 5
n_data = 5

models = ['TGAN', 'CTGAN']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        models.append('{}_{}'.format(i,j))

tmp = []
        
for i in range(n_models):
    for m in models:
        tmp.append(m + '_{:0>2d}'.format(i+1))
        
tmp.sort()
models = tmp

In [5]:
df = pd.read_csv('../data/' + dataset + '/data.csv')

In [6]:
input_folder = '../output/' + dataset + '/'
output_folder = '../synth_data/' + dataset + '/'

In [7]:
def is_a_DATGAN(name):
    if 'TGAN' in name or 'CTGAN' in name:
        return False
    else:
        return True
    
def round_some_vars(samples, dataset):
    if dataset is 'Chicago':
        samples.age = np.round(samples.age)
    elif dataset is 'LPMC':
        samples.age = np.round(samples.age)
        samples.distance = np.round(samples.distance)
        
    return samples

In [8]:
for i, m in enumerate(models):
    
    name = m.split('_')[0]
    
    print("Sampling synthetic data from model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

    
    model = None

    if 'CTGAN' in name:
        import ctgan
        from ctgan import CTGANSynthesizer
        
        ctgan = CTGANSynthesizer()
        model = ctgan.load(input_folder + m + '/trained.pickle')

    elif 'TGAN' in name:
        from tgan.model import TGANModel
        model = TGANModel.load(input_folder + m + '/trained.pickle')
    else:

        LIB = None

        if 'WGAN' in name:
            from modules.datgan import DATWGAN as LIB
        if any(x in name for x in ('WGANGP','WGGP')) :
            from modules.datgan import DATWGANGP as LIB
        elif 'SGAN' in name:
            from modules.datgan import DATSGAN as LIB
        else:
            from modules.datgan import DATWGAN as LIB

        model = LIB.load(input_folder + m + '/', 'trained')
    
    for k in range(n_data):
            
        if is_a_DATGAN(name):   
            for samp_type in ['NO', 'BO', 'OD']:
                samples = model.sample(len(df), samp_type)
                
                samples = round_some_vars(samples, dataset)
                
                if n_data == 1:
                    samples.to_csv(output_folder + m + '_{}.csv'.format(samp_type), index=False)
                else:
                    samples.to_csv(output_folder + m + '_{}_{:0>2d}.csv'.format(samp_type, k+1), index=False)
        else:
            samples = model.sample(len(df))

            samples = round_some_vars(samples, dataset)

            if n_data == 1:
                samples.to_csv(output_folder + m + '.csv', index=False)
            else:
                samples.to_csv(output_folder + m + '_{:0>2d}.csv'.format(k+1), index=False)
                
print("\033[1mFINISHED!\033[0m")

Sampling synthetic data from model CTGAN_01 (1/55)
Sampling synthetic data from model CTGAN_02 (2/55)
Sampling synthetic data from model CTGAN_03 (3/55)
Sampling synthetic data from model CTGAN_04 (4/55)
Sampling synthetic data from model CTGAN_05 (5/55)
Sampling synthetic data from model SGAN_OR_01 (6/55)














Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1123 12:55:57 @DATSGANModel.py:205] Creating cell for age (in-edges: 0; ancestors: 0)

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1123 12:55:57 @registry.py:126] gen/LSTM/age/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1123 12:55:57 @registry.py:134] gen/LSTM/age/FC output: [500, 50]
[1123 12:55:57 @registry.py:126] gen/LSTM/age/FC_val input: [500

[1123 12:55:57 @registry.py:126] gen/LSTM/education_level/FC_input input: [500, 6]
[1123 12:55:57 @registry.py:134] gen/LSTM/education_level/FC_input output: [500, 100]
[1123 12:55:57 @DATSGANModel.py:205] Creating cell for work_status (in-edges: 2; ancestors: 3)
[1123 12:55:57 @registry.py:126] gen/LSTM/concat-work_status/FC_inputs input: [500, 200]
[1123 12:55:57 @registry.py:134] gen/LSTM/concat-work_status/FC_inputs output: [500, 100]
[1123 12:55:57 @registry.py:126] gen/LSTM/concat-work_status/FC_states input: [500, 200]
[1123 12:55:57 @registry.py:134] gen/LSTM/concat-work_status/FC_states output: [500, 100]
[1123 12:55:57 @registry.py:126] gen/LSTM/concat-work_status/FC_h_outputs input: [500, 200]
[1123 12:55:57 @registry.py:134] gen/LSTM/concat-work_status/FC_h_outputs output: [500, 100]
[1123 12:55:57 @registry.py:126] gen/LSTM/work_status/FC input: [500, 100]
[1123 12:55:57 @registry.py:134] gen/LSTM/work_status/FC output: [500, 50]
[1123 12:55:57 @registry.py:126] gen/LSTM/w

[1123 12:55:58 @registry.py:134] gen/LSTM/hh_vehicles/FC_prob output: [500, 9]
[1123 12:55:58 @registry.py:126] gen/LSTM/hh_vehicles/FC_input input: [500, 9]
[1123 12:55:58 @registry.py:134] gen/LSTM/hh_vehicles/FC_input output: [500, 100]
[1123 12:55:58 @DATSGANModel.py:205] Creating cell for hh_bikes (in-edges: 2; ancestors: 6)
[1123 12:55:58 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_inputs input: [500, 200]
[1123 12:55:58 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_inputs output: [500, 100]
[1123 12:55:58 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_states input: [500, 200]
[1123 12:55:58 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_states output: [500, 100]
[1123 12:55:58 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_h_outputs input: [500, 200]
[1123 12:55:58 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_h_outputs output: [500, 100]
[1123 12:55:58 @registry.py:126] gen/LSTM/hh_bikes/FC input: [500, 100]
[1123 12:55:58 @registry.py:134] gen/LSTM/hh_bikes/FC output: [500, 50]

|                                                                                          |17/?[00:01<00:00,15.24it/s]

[1123 12:56:02 @DATSGAN.py:266] Loading Preprocessor!


...

[1123 13:11:17 @DATSGAN.py:266] Loading Preprocessor!



|                                                                                          |17/?[00:00<00:00,23.30it/s]

[1123 13:11:18 @DATSGAN.py:266] Loading Preprocessor!



|                                                                                          |17/?[00:00<00:00,23.47it/s]


FINISHED!
